In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib import pyplot as plt
plt.rcParams['axes.grid'] = True

In [9]:
hdat = pd.read_csv("../history.csv")
hdat.head()

,DATE_OF_INTEREST,AS_OF,"Cases/day, 7 day avg",DEATH_COUNT,"Deaths/day, 7 day avg",HOSPITALIZED_CASE_COUNT,"Hospitalized/day, 7 day avg",NEW_COVID_CASE_COUNT
0,2020-03-02,2020-03-26 11:26:41,1.00,NaN,NaN,NaN,NaN,1
1,2020-03-03,2020-03-26 11:26:41,1.50,NaN,NaN,3.0,3.000000,2
2,2020-03-04,2020-03-26 11:26:41,4.00,NaN,NaN,3.0,3.000000,9
3,2020-03-05,2020-03-26 11:26:41,3.25,NaN,NaN,4.0,3.333333,1
4,2020-03-06,2020-03-26 11:26:41,4.20,NaN,NaN,5.0,3.750000,8


In [10]:
hdat["DATE_OF_INTEREST"] = pd.to_datetime(hdat["DATE_OF_INTEREST"])
hdat["AS_OF"] = pd.to_datetime(hdat["AS_OF"])
hdat["Days"] = hdat.groupby("DATE_OF_INTEREST").cumcount()
hdat["Age"] = hdat["AS_OF"] - hdat["DATE_OF_INTEREST"]

In [11]:
hdat.head()

,DATE_OF_INTEREST,AS_OF,"Cases/day, 7 day avg",DEATH_COUNT,"Deaths/day, 7 day avg",HOSPITALIZED_CASE_COUNT,"Hospitalized/day, 7 day avg",NEW_COVID_CASE_COUNT,Days,Age
0,2020-03-02,2020-03-26 11:26:41,1.00,NaN,NaN,NaN,NaN,1,0,24 days 11:26:41
1,2020-03-03,2020-03-26 11:26:41,1.50,NaN,NaN,3.0,3.000000,2,0,23 days 11:26:41
2,2020-03-04,2020-03-26 11:26:41,4.00,NaN,NaN,3.0,3.000000,9,0,22 days 11:26:41
3,2020-03-05,2020-03-26 11:26:41,3.25,NaN,NaN,4.0,3.333333,1,0,21 days 11:26:41
4,2020-03-06,2020-03-26 11:26:41,4.20,NaN,NaN,5.0,3.750000,8,0,20 days 11:26:41


In [12]:
hdat["Days"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [14]:
firstRpt = hdat[["DATE_OF_INTEREST", "NEW_COVID_CASE_COUNT"]].groupby("DATE_OF_INTEREST").first()
firstRpt.rename(columns={"NEW_COVID_CASE_COUNT": "First"}, inplace=True)
hdat = hdat.merge(firstRpt, how="left", on="DATE_OF_INTEREST")
hdat.head()

,DATE_OF_INTEREST,AS_OF,"Cases/day, 7 day avg",DEATH_COUNT,"Deaths/day, 7 day avg",HOSPITALIZED_CASE_COUNT,"Hospitalized/day, 7 day avg",NEW_COVID_CASE_COUNT,Days,Age,First
0,2020-03-02,2020-03-26 11:26:41,1.00,NaN,NaN,NaN,NaN,1,0,24 days 11:26:41,1
1,2020-03-03,2020-03-26 11:26:41,1.50,NaN,NaN,3.0,3.000000,2,0,23 days 11:26:41,2
2,2020-03-04,2020-03-26 11:26:41,4.00,NaN,NaN,3.0,3.000000,9,0,22 days 11:26:41,9
3,2020-03-05,2020-03-26 11:26:41,3.25,NaN,NaN,4.0,3.333333,1,0,21 days 11:26:41,1
4,2020-03-06,2020-03-26 11:26:41,4.20,NaN,NaN,5.0,3.750000,8,0,20 days 11:26:41,8


In [15]:
lastRpt = hdat[["DATE_OF_INTEREST", "NEW_COVID_CASE_COUNT"]].groupby("DATE_OF_INTEREST").last()
lastRpt.rename(columns={"NEW_COVID_CASE_COUNT": "Last"}, inplace=True)
hdat = hdat.merge(lastRpt, how="left", on="DATE_OF_INTEREST")
hdat.head()

,DATE_OF_INTEREST,AS_OF,"Cases/day, 7 day avg",DEATH_COUNT,"Deaths/day, 7 day avg",HOSPITALIZED_CASE_COUNT,"Hospitalized/day, 7 day avg",NEW_COVID_CASE_COUNT,Days,Age,First,Last
0,2020-03-02,2020-03-26 11:26:41,1.00,NaN,NaN,NaN,NaN,1,0,24 days 11:26:41,1,1
1,2020-03-03,2020-03-26 11:26:41,1.50,NaN,NaN,3.0,3.000000,2,0,23 days 11:26:41,2,1
2,2020-03-04,2020-03-26 11:26:41,4.00,NaN,NaN,3.0,3.000000,9,0,22 days 11:26:41,9,5
3,2020-03-05,2020-03-26 11:26:41,3.25,NaN,NaN,4.0,3.333333,1,0,21 days 11:26:41,1,3
4,2020-03-06,2020-03-26 11:26:41,4.20,NaN,NaN,5.0,3.750000,8,0,20 days 11:26:41,8,7


In [16]:
hdat

,DATE_OF_INTEREST,AS_OF,"Cases/day, 7 day avg",DEATH_COUNT,"Deaths/day, 7 day avg",HOSPITALIZED_CASE_COUNT,"Hospitalized/day, 7 day avg",NEW_COVID_CASE_COUNT,Days,Age,First,Last
0,2020-03-02,2020-03-26 11:26:41,1.000000,NaN,NaN,NaN,NaN,1,0,24 days 11:26:41,1,1
1,2020-03-03,2020-03-26 11:26:41,1.500000,NaN,NaN,3.0,3.000000,2,0,23 days 11:26:41,2,1
2,2020-03-04,2020-03-26 11:26:41,4.000000,NaN,NaN,3.0,3.000000,9,0,22 days 11:26:41,9,5
3,2020-03-05,2020-03-26 11:26:41,3.250000,NaN,NaN,4.0,3.333333,1,0,21 days 11:26:41,1,3
4,2020-03-06,2020-03-26 11:26:41,4.200000,NaN,NaN,5.0,3.750000,8,0,20 days 11:26:41,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...
926,2020-04-14,2020-04-19 13:25:22,3758.142857,311.0,399.000000,663.0,895.285714,3277,5,5 days 13:25:22,26,3277
927,2020-04-15,2020-04-19 13:25:22,3387.857143,261.0,370.714286,540.0,783.142857,2745,3,4 days 13:25:22,34,2745
928,2020-04-16,2020-04-19 13:25:22,2926.142857,196.0,337.142857,218.0,664.142857,1573,2,3 days 13:25:22,26,1573
929,2020-04-17,2020-04-19 13:25:22,2410.428571,94.0,291.714286,29.0,531.000000,560,1,2 days 13:25:22,38,560


In [31]:
growth = hdat["last"]/hdat["NEW_COVID_CASE_COUNT"]

In [26]:
df = hdat.pivot_table(index="Age",
                 columns="DATE_OF_INTEREST",
                 values="growth")

In [27]:
hdat["growth"].unique()

array([nan])

In [28]:
hdat["last"] = hdat[["DATE_OF_INTEREST", "NEW_COVID_CASE_COUNT"]].groupby("DATE_OF_INTEREST").last()

In [29]:
hdat["last"].unique()

array([nan])

In [30]:
hdat[["DATE_OF_INTEREST", "NEW_COVID_CASE_COUNT"]].groupby("DATE_OF_INTEREST").transform(lambda x: x.last())

TypeError: Transform function invalid for data types